In [2]:
#import required packages
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

#Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

#Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
    'archive/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical")

#Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
    'archive/test',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical")

#Create model structure
emotion_detection_model = Sequential()

emotion_detection_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_detection_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_detection_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_detection_model.add(Dropout(0.25))

emotion_detection_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_detection_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_detection_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_detection_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_detection_model.add(Dropout(0.25))

emotion_detection_model.add(Flatten())
emotion_detection_model.add(Dense(1024, activation='relu'))
emotion_detection_model.add(Dropout(0.5))
emotion_detection_model.add(Dense(7, activation='softmax'))

emotion_detection_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

# Train the neural network/model
emotion_detection_model_info = emotion_detection_model.fit(
        train_generator,
        y=None,
        steps_per_epoch=28709 // 64,
        epochs=80,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        )

# save model structure in json file
model_json = emotion_detection_model.to_json()
with open("emotion_detection_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_detection_model.save_weights('emotion_detection_model.h5')

# converting the model to TF lite
converter = tf.lite.TFLiteConverter.from_keras_model(emotion_detection_model.h5)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f


Found 45579 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/80
448/448 [==============================] - 181s 403ms/step - loss: 1.7446 - accuracy: 0.3178 - val_loss: 1.7667 - val_accuracy: 0.2740
Epoch 2/80
448/448 [==============================] - 191s 426ms/step - loss: 1.5800 - accuracy: 0.3923 - val_loss: 1.5724 - val_accuracy: 0.3908
Epoch 3/80
448/448 [==============================] - 182s 407ms/step - loss: 1.4742 - accuracy: 0.4388 - val_loss: 1.4978 - val_accuracy: 0.4265
Epoch 4/80
448/448 [==============================] - 186s 415ms/step - loss: 1.3987 - accuracy: 0.4740 - val_loss: 1.4314 - val_accuracy: 0.4474
Epoch 5/80
448/448 [==============================] - 192s 428ms/step - loss: 1.3454 - accuracy: 0.4933 - val_loss: 1.3967 - val_accuracy: 0.4639
Epoch 6/80
448/448 [==============================] - 196s 437ms/step - loss: 1.2946 - accuracy: 0.5105 - val_loss: 1.3348 - val_accuracy: 0.4862
Epoch 7/80
448/448 [===================

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model()
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)